In [1]:
import yaml
from datacreator import StockData,EconomyData,FewShot_Sentiment_Generator
from openbb_terminal.sdk import openbb
from langchain.embeddings import OpenAIEmbeddings
from scipy.spatial import distance
from serpapi import GoogleSearch
import logging
logging.disable()
# Open the YAML file and load its contents into a Python object
with open('apis.yaml', 'r') as file:
    yaml_data = yaml.load(file, Loader=yaml.FullLoader)
from langchain.vectorstores import FAISS
# # Convert the YAML data to a dictionary
data_dict = dict(yaml_data)
openbb.keys.av(key = data_dict['OPENBB']['ALPHA_VANTAGE_KEY'], persist = True)
openbb.keys.fmp(key = data_dict['OPENBB']['FINANCIALMODELLING_AND_PREP_KEY'],persist = True)
openbb.keys.polygon(key = data_dict['OPENBB']['POLYGON_KEY'],persist = True)
openbb.keys.finnhub(key = data_dict['OPENBB']['FINNHUB_KEY'],persist = True)
openbb.keys.fred(key = data_dict['OPENBB']['FRED_KEY'],persist = True)
SERP_API_KEY = data_dict['GOOGLESEARCH']

Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://docs.openbb.co/sdk/quickstart/installation/

Saving log to /Users/maneelreddy/Downloads/Entreprenuer/lstocklopenbbm/data/logs



In [2]:
import time
for stock in ['AAPL','TSLA','GOOG']:
    stock_data = StockData(stock)
    stock_data.initialize_folders()
    stock_data.ratio_file()
    stock_data.cash_file()
    stock_data.balance_file()
    stock_data.est_file()
    stock_data.fraud_file()
    stock_data.income_file()
    stock_data.analysis_file()
    stock_data.news_file()
    time.sleep(1)

In [ ]:
df = openbb.stocks.fa.est('AAPL')[0]

In [ ]:
df = openbb.stocks.fa.income('AAPL')

In [ ]:
with open('sample.csv','r') as f:
    x = f.read()

In [ ]:
print(x)

In [ ]:
import yaml
from datacreator import StockData, EconomyData
from openbb_terminal.sdk import openbb


# Open the YAML file and load its contents into a Python object
with open("apis.yaml", "r") as file:
    yaml_data = yaml.load(file, Loader=yaml.FullLoader)

# Convert the YAML data to a dictionary
data_dict = dict(yaml_data)
openbb.keys.av(key=data_dict["OPENBB"]["ALPHA_VANTAGE_KEY"], persist=True)
openbb.keys.fmp(
    key=data_dict["OPENBB"]["FINANCIALMODELLING_AND_PREP_KEY"], persist=True
)
openbb.keys.polygon(key=data_dict["OPENBB"]["POLYGON_KEY"], persist=True)
openbb.keys.finnhub(key=data_dict["OPENBB"]["FINNHUB_KEY"], persist=True)
openbb.keys.fred(key=data_dict["OPENBB"]["FRED_KEY"], persist=True)
open_ai_params = {
    "max_tokens": 512,
    "openai_api_key": yaml_data["LLMS"]["OPENAI_API_KEY"],
}
stocks = yaml_data['STOCKS']

# for stock in stocks:
#     stock_data = StockData(stock)
#     stock_data.initialize_folders()
#     stock_data.analysis_file()
#     stock_data.ratio_file()
#     stock_data.cash_file()
#     stock_data.est_file()
#     stock_data.fraud_file()
#     stock_data.income_file()
#     stock_data.balance_file()
#     stock_data.news_file()


In [ ]:
class Sentiment_Generator:
    def __init__(self,embeddings):
        self.embeddings = embeddings
    
    def collect_news_files(self):
        main_file = []
        for stock in os.listdir('../ticker'):
            try:
                df = pd.read_csv(f'../ticker/{stock}/news/c_news.csv')
                df.insert(0,'ticker',stock)
                df = df[['ticker','headline','datetime']]
            except:
                pass
            main_file.append(df)
        self.document_file = pd.concat(main_file)
    
    def get_scores(self):
        docs = list(self.document_file['headline'])
        classifications = ['Strongly Positive','Positive','Neutral','Negative','Strongly Negative']
        ## Create a faiss vector database
        #import pdb;pdb.set_trace()
        faiss_classifications = FAISS.from_texts(classifications,self.embeddings)
        self.similarity_scores = []
        for x in docs:
            #import pdb;pdb.set_trace()
            self.similarity_scores.append(faiss_classifications.similarity_search_with_score(x,k = 1)[0][0].page_content)
        self.document_file['similarity'] = self.similarity_scores
        
    def save_scores(self):
        self.document_file.to_csv('output/sentiment_scores.csv')
        

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key = yaml_data["LLMS"]["OPENAI_API_KEY"])
sentiment_generation = Sentiment_Generator(embeddings)
sentiment_generation.collect_news_files()
sentiment_generation.get_scores()

In [ ]:
sentiment_generation.to_csv('news.csv')

In [ ]:
sentiment_generation.document_file.to_csv('news.csv')

In [ ]:
sentiment_generation.document_file.to_csv('sentiment_scores.csv')

In [ ]:
sentiment_generation.document_file.similarity.head()